## ZBTCNN TANH
Predicts percentage price increase  
Uses technicals aswell from csv_formatter.ipynb

In [2]:
!source /etc/profile

import os
import sys
import random
import time
from collections import deque
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Activation, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import optimizers
from tensorflow.keras import utils
import joblib

# Check GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.config.list_physical_devices()

Num GPUs Available:  1


2022-02-07 06:17:04.563111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 06:17:04.571505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 06:17:04.571857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
SEQ_LEN = 24 #hours
FUTURE_PERIOD_PREDICT = 1 #hours

# Model parameters
EPOCHS = 20
BATCH_SIZE = 64
NAME = f"TANH-e{EPOCHS}-b{BATCH_SIZE}-s{SEQ_LEN}-fpp{FUTURE_PERIOD_PREDICT}-{int(time.time())}"

In [5]:
## Import data
# DATA MUST BE FORMATTED USING CSV_FORMATTER.IPYNB

csv_file = "data/formatted/BTCUSDT-1h-data.csv"

data = pd.read_csv(csv_file, skiprows=[0], names=["timestamp", "open", "high", "low", "close", "volume", "rsi", "ema"])

data.set_index("timestamp", inplace=True)

data.head()

,open,high,low,close,volume,rsi,ema
timestamp,,,,,,,
1503064800,4304.15,4371.52,4296.04,4356.31,51.563675,52.623958,4327.151560
1503068400,4356.31,4357.37,4302.72,4340.31,24.093449,51.678528,4327.804777
1503072000,4320.52,4340.31,4287.79,4331.71,15.118957,51.167386,4327.995329
1503075600,4302.97,4318.16,4221.05,4293.09,46.533767,48.919621,4326.319858
1503079200,4293.09,4293.09,4193.70,4259.40,74.368943,47.054235,4323.157459


In [19]:
## Formatting data

def classify(current, future):
    return float((future - current) / current)


data["future"] = data["close"].shift(-FUTURE_PERIOD_PREDICT)

# Cut off NaNs
# data = data[:-FUTURE_PERIOD_PREDICT]
data.dropna(inplace=True)

data["target"] = list(map(classify, data["close"], data["future"]))
# data[["close", "future", "target"]].tail()
data = data.drop("future", 1)

# Split dataset
last_5_pct = int(len(data) * .95)

train_data = data[:last_5_pct]
validation_data = data[last_5_pct:]

print(f"{len(train_data)} :: {len(validation_data)}")

data.head()

36695 :: 1932


/tmp/ipykernel_14263/1604648129.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop("future", 1)


,open,high,low,close,volume,rsi,ema,target
timestamp,,,,,,,,
1503064800,4304.15,4371.52,4296.04,4356.31,51.563675,52.623958,4327.151560,-0.003673
1503068400,4356.31,4357.37,4302.72,4340.31,24.093449,51.678528,4327.804777,-0.001981
1503072000,4320.52,4340.31,4287.79,4331.71,15.118957,51.167386,4327.995329,-0.008916
1503075600,4302.97,4318.16,4221.05,4293.09,46.533767,48.919621,4326.319858,-0.007847
1503079200,4293.09,4293.09,4193.70,4259.40,74.368943,47.054235,4323.157459,-0.005285


In [7]:
## Helper
## Ratios of buy to sell targets

## See how balances the input data is

sell_counter = len([x for x in list(data["target"]) if x < 0])
buy_counter = len([x for x in list(data["target"]) if x > 0])

pct_sell = sell_counter / len(data)
pct_buy = buy_counter / len(data)

print(f"{pct_sell} :: {pct_buy}")

KeyError: 'target'

In [21]:
## Preprocess Data

def preprocess_df(df_p):

    df = pd.DataFrame()
    for col in df_p.columns:
        df[col] = df_p[col]

    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.replace([np.inf, -np.inf], np.nan, inplace=True)
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col])
        else:
            scaler = preprocessing.Sta
            df[col] = scaler.fit_transform(np.array(df[col]).reshape(-1, 1))
            joblib.dump(scaler, f"scalers/{NAME}")

    df.dropna(inplace=True)

    sequential_data = []
    prev_periods = deque(maxlen=SEQ_LEN)

    for i in df.values:
        prev_periods.append([n for n in i[:-1]])
        if len(prev_periods) == SEQ_LEN:
            sequential_data.append([np.array(prev_periods), i[-1]])

    # random.shuffle(sequential_data)

    # Balance buys and sells
    buys = []
    sells = []

    for seq, target in sequential_data:

        if target < 0.0:
            sells.append([seq, target])
        elif target > 0.0:
            buys.append([seq, target])

    lower = min(len(buys), len(sells))

    buys = buys[:lower]
    sells = sells[:lower]

    sequential_data = buys + sells

    random.shuffle(sequential_data)

    X = [d[0] for d in sequential_data]
    Y = [d[1] for d in sequential_data]

    return np.array(X), np.array(Y)
    

train_x, train_y = preprocess_df(train_data)
validation_x, validation_y = preprocess_df(validation_data)

In [22]:
## Dataset metrics

print(f"Train : Validation == {len(train_x)} : {len(validation_x)}")
print(f"Train\t\tBuys : Sells == {len([x for x in list(train_y) if x > 0])} : {len([x for x in list(train_y) if x < 0])}")
print(f"Validation\tBuys : Sells == {len([x for x in list(validation_y) if x > 0])} : {len([x for x in list(validation_y) if x < 0])}")

Train : Validation == 36568 : 1898
Train		Buys : Sells == 18284 : 18284
Validation	Buys : Sells == 949 : 949


In [23]:
## Make model

model = Sequential()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)


model.add(LSTM(128, input_shape=(train_x.shape[1:]), activation="tanh", return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(train_x.shape[1:]), activation="tanh", return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(train_x.shape[1:]), activation="tanh"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(1, activation="tanh"))



model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=optimizer, metrics=["mse"])

tb = TensorBoard(log_dir=f"logs/{NAME}")

filepath = NAME + "-e{epoch:02d}-vacc{mse:.3f}-" + str(int(time.time()))
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="mse", verbose=1, save_best_only=True, mode="max"))

2022-02-07 04:51:41.623429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 04:51:41.623911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 04:51:41.624313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 04:51:41.956431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-07 04:51:41.956842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [24]:
with tf.device("/device:GPU:0"):
    history = model.fit(x=train_x, y=train_y,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(validation_x, validation_y),
                        callbacks=[tb, checkpoint]
                        )

Epoch 1/20


2022-02-07 04:51:53.205065: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204


571/572 [============================>.] - ETA: 0s - loss: 1.1815 - mse: 1.1815

2022-02-07 04:52:12.903380: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/TANH-e20-b64-s24-fpp1-1644209479-e01-vacc1.181-1644209502.model/assets


...

INFO:tensorflow:Assets written to: models/TANH-e20-b64-s24-fpp1-1644209479-e19-vacc1.000-1644209502.model/assets


572/572 [==============================] - 25s 44ms/step - loss: 1.0004 - mse: 1.0004 - val_loss: 1.0162 - val_mse: 1.0162
Epoch 20/20
571/572 [============================>.] - ETA: 0s - loss: 1.0006 - mse: 1.0006

INFO:tensorflow:Assets written to: models/TANH-e20-b64-s24-fpp1-1644209479-e20-vacc1.001-1644209502.model/assets


INFO:tensorflow:Assets written to: models/TANH-e20-b64-s24-fpp1-1644209479-e20-vacc1.001-1644209502.model/assets


572/572 [==============================] - 25s 44ms/step - loss: 1.0006 - mse: 1.0006 - val_loss: 1.0100 - val_mse: 1.0100


In [25]:
from pushbullet import Pushbullet
pb = Pushbullet("o.nyntgspLep97yl0oPDbp0nAbMIDUGiO5")
push = pb.push_note(f"{time.asctime()}", "ML Training Done")